<a href="https://colab.research.google.com/github/GreihMurray/NLP-3/blob/Semi_Murray/semisupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Reshape
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import joblib
from sklearn.metrics import accuracy_score
import pickle
import nltk
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
from sklearn.semi_supervised import LabelPropagation
from numpy import concatenate

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Original

In [ ]:
def read_file_to_sents():
    all_data = []
    with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/train.tsv", encoding="utf-8") as file:
        f = csv.reader(file, delimiter="\t")
        for line in tqdm(f, desc="Reading data..."):
            word = line[0]
            graphemes = line[1].split('-')

            cur_word = []

            for i in range(0, len(graphemes)):
                for j in range(0, len(graphemes[i])):
                    if j == 0:
                        cur_word.append((graphemes[i][j], 'B'))
                    else:
                        cur_word.append((graphemes[i][j], 'I'))

            all_data.append(cur_word)

    return all_data

Original

In [ ]:
def split_data(data):
    split_x = []
    split_y = []
    
    for word in data:
        cur_x = []
        cur_y = []
        for letter in word:
            cur_x.append(letter[0])
            cur_y.append(letter[1])

        split_x.append(cur_x)
        split_y.append(cur_y)

    return split_x, split_y

original

In [ ]:
def combine_all_y(y):
    all_y = []

    for entry in y:
        for letter in entry:
            all_y.append(letter)

    return all_y

original

In [ ]:
def split_out(x, y):
    fixed_y = []

    for j in range(0, len(x)):
        cur_word = []
        for i in range(0, len(x[j])):
            cur_word.append(y[i])
        fixed_y.append(cur_word)

    return fixed_y
        

Based on code from https://machinelearningmastery.com/semi-supervised-learning-with-label-propagation/

In [31]:
def semi_supervised():
    data = read_file_to_sents()

    x, y = split_data(data)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 50)

    x_lab = combine_all_y(x_train[:650])
    y_lab = combine_all_y(y_train[:650])
    x_train = combine_all_y(x_train[650:])

    le = LabelEncoder()

    le.fit(y_lab)
    y_hold = le.transform(y_lab)

    x_train_mixed = concatenate((x_lab, x_train))
    fle = LabelEncoder()
    fle.fit(x_train_mixed)
    x_train_mixed = fle.transform(x_train_mixed)
    x_train_mixed = x_train_mixed.reshape(-1, 1)

    nolabel = [-1 for _ in range(len(y_hold), len(x_train_mixed))]

    y_train_mixed = concatenate((y_hold, nolabel))

    print(len(x_train_mixed))
    print(len(y_train_mixed))

    model = LabelPropagation()

    model.fit(x_train_mixed, y_train_mixed)

    x_test = combine_all_y(x_test)
    x_test = fle.transform(x_test)
    x_test = x_test.reshape(-1, 1)

    y_hat = model.predict(x_test)
    y_test = combine_all_y(y_test)
    y_test = le.transform(y_test)
    y_test = y_test.reshape(-1, 1)


    score = accuracy_score(y_test, y_hat)

    print(score*100)

Original

In [32]:
semi_supervised()

Reading data...: 12812it [00:00, 203356.73it/s]


43606
43606
96.95180557462584
